## Initialization

In [116]:
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11

In [117]:
import numpy as np
import numba
import pandas as pd
from reactionmodel.model import Species, Reaction, Model

In [118]:
def make_df(results, t_eval):
    result_series = []

    for r in results:
        t, y = r.restricted_values(t_eval)
        result_series.append(pd.Series(y.T.squeeze(), index=t))
    
    df = pd.concat(result_series, axis=1)
    df = df.dropna()
    return df

# ABSORPTION

So as an approximate rule of thumb, the absorption exponential generally becomes negligible and ln(Cp) values fall on a terminal line when t > 7·t½,abs.

https://www.sciencedirect.com/topics/medicine-and-dentistry/absorption-half-life#:~:text=First%2Dorder%20absorption%20occurs%20when,at%20the%20site%20of%20administration.

$$ \lambda(t) = \frac{R_0 \mu}{1 + (1/\text{IC}_{50} e^{- \log(2) t / t_{\text{half}}})^{m_\text{drug}}} $$

First order absorption:

$$r_{\text{abs}} = k_a FD \cdot e^{-k_a t} $$

Concentration as a function of time:

$$ C(t) = FD [ 1 - e^{-k_a t }]

In [119]:
mu = 1.0
R0 = 3.0
R0_min = 0.2
t_intervention = 2.0
t_to_min = 3.0

early_start = False

@numba.jit(nopython=True)
def lambda_t_early_start(t):
    if t < t_intervention:
        return R0 * mu
    if t > t_intervention + t_to_min:
        return R0_min * mu

    return R0 * mu - (t - t_intervention) * (R0 - R0_min) * mu / t_to_min

@numba.jit(nopython=True)
def lambda_t_late_start(t):
    return lambda_t_early_start(t+t_intervention)

if early_start:
    lambda_t = lambda_t_early_start
else:
    lambda_t = lambda_t_late_start

In [120]:
t_upper = (t_intervention + t_to_min) * 3

t_eval_late_start = np.linspace(0.0, t_upper - t_intervention, int((t_upper-t_intervention)*10+1))
t_eval_early_start = np.linspace(0.0, t_upper, int((t_upper)*10+1))
t_span_late_start = [0, t_upper]
t_span_early_start = [0, t_upper + 1]

if early_start:
    t_eval = t_eval_early_start
    t_span = t_span_early_start
else:
    t_eval = t_eval_late_start
    t_span = t_span_late_start

R0s = [lambda_t_early_start(t)/mu for t in t_eval_early_start]

In [ ]:
fig, ax = plt.subplots(1)

ax.plot(t_eval_early_start, R0s)
ax.set_ylabel('R0')
plt.axhline(1.0, c='r', linestyle='--', alpha=0.3)
ax.set_xlabel('time (lifespans of infected cell)')

In [122]:
y0_early_start = 5
if early_start:
    discontinuities = [t_intervention, t_intervention+t_to_min]
else:
    discontinuities = [t_to_min]

Y = Species('Y')

death = Reaction([Y], [], k=mu)
birth = Reaction([Y], [(Y, 2)], k=lambda_t)

m = Model([Y], [birth, death])
m_early_start = Model([Y], [Reaction([Y], [(Y, 2)], k=lambda_t_early_start), death])

In [123]:
epsilon = 0.05

## Deterministic

In [ ]:
from scipy.integrate import solve_ivp
dydt = m_early_start.get_dydt(jit=True)

det_result = solve_ivp(dydt, t_span_early_start, m_early_start.make_initial_condition({'Y':y0_early_start}), t_eval=t_eval_early_start, args=(0,))
det_result = pd.Series(det_result.y.T.squeeze(), index=det_result.t)
det_result.name = 'deterministic'

ax = det_result.plot()
det_result.index = det_result.index.round(3)

ax.axvline(1.0, c='r', linestyle='--')
ax.axhline(10000, c='r', linestyle='--')

In [125]:
ic = m.make_initial_condition({'Y':det_result.loc[t_intervention]})

## Homogeneous

In [ ]:
from hybrid.tau import TauLeapSimulator

tau_simulator = TauLeapSimulator(
    m.get_k(jit=True),
    m.stoichiometry(),
    m.kinetic_order(),
    epsilon=epsilon,
    discontinuities=discontinuities
    #time_handling='inhomogeneous_monotonic'
)

In [ ]:
results = tau_simulator.run_simulations(
    100,
    t_span,
    ic,
    np.random.default_rng(),
    t_eval=t_eval,
)

In [ ]:
results[0].plot(m.legend())

In [129]:
tau_df = make_df(results, t_eval)
tau_df.index = tau_df.index.round(3)

In [130]:
cmap = {4:'gold', 2:'dodgerblue', 0: 'r', -2:'b', 1:'r'}

In [131]:
step_df_homo = pd.DataFrame({'t': results[0].step_t_history, 'status': results[0].status_history, 'y': results[0].step_y_history[0]})
step_df_homo['step size'] = step_df_homo['t'].diff()
step_df_homo['step amount'] = step_df_homo['y'].diff()
step_df_homo = step_df_homo.loc[1:]
step_df_homo['t'] += t_intervention

In [ ]:
color = step_df_homo.status.map(cmap)
step_df_homo.plot.scatter(x='t', y='step size', c=color, ylim=[0.0, 0.1], alpha=0.6)

## Inbetween

In [133]:
from hybrid.tau import TauLeapSimulator

inhom_tau_simulator = TauLeapSimulator(
    m.get_k(jit=True),
    m.stoichiometry(),
    m.kinetic_order(),
    epsilon=epsilon*2,
    time_handling='inhomogeneous_monotonic_homogeneous_gillespie',
    discontinuities=discontinuities,
)

In [ ]:
ibresults = inhom_tau_simulator.run_simulations(
    50,
    t_span,
    ic,
    np.random.default_rng(0),
    t_eval=t_eval
)

In [135]:
in_between_df = make_df(ibresults, t_eval)
in_between_df.index = in_between_df.index.round(3)

In [136]:
step_df_ib = pd.DataFrame({'t': ibresults[0].step_t_history, 'status': ibresults[0].status_history, 'y': ibresults[0].step_y_history[0]})
step_df_ib['step size'] = step_df_ib['t'].diff()
step_df_ib['step amount'] = step_df_ib['y'].diff()
step_df_ib = step_df_ib.loc[1:]
step_df_ib['t'] += t_intervention

In [ ]:
step_df_ib['step size'].max()

In [ ]:
upper = 0.2
cmap = {4:'seagreen', 2:'seagreen', 0: 'r', -2:'b', 1:'r'}
ax = step_df_ib.plot.scatter(x='t', y='step size', c=step_df_ib.status.map(cmap), ylim=[0.0, upper], alpha=0.5)
cmap = {4:'orange', 2:'orange', 0: 'r', -2:'b', 1:'r'}
step_df_homo.plot.scatter(x='t', y='step size', c=step_df_homo.status.map(cmap), ylim=[0.0, upper], alpha=0.4, ax=ax)
ax.set_xlabel('time (infectious periods)')
plt.savefig('./inhomogeneous_stepsize.png', dpi=300)

## Implicit

In [ ]:
from hybrid.tau import TauLeapSimulator

implicit_tau_simulator = TauLeapSimulator(
    m.get_k(jit=True),
    m.stoichiometry(),
    m.kinetic_order(),
    epsilon=epsilon,
    discontinuities=discontinuities,
    method='implicit'
)

In [ ]:
impresults = implicit_tau_simulator.run_simulations(
    50,
    t_span,
    ic,
    np.random.default_rng(),
    t_eval=t_eval
)

In [141]:
implicit_df = make_df(impresults, t_eval)
implicit_df.index = implicit_df.index.round(3)

## Results

In [ ]:
fig, ax = plt.subplots(3, height_ratios=[1.0, 0.6, 0.6], figsize=(7,11), sharex=True)


data = {
    #'gillespie': gillespie_df.mean(axis=1),
    'conventional $\\tau$': tau_df.mean(axis=1),
    #'inhomogeneous $\\tau$': inhomo_df.mean(axis=1),
    'semi-inhomogeneous $\\tau$': in_between_df.mean(axis=1),
    'implicit $\\tau$': implicit_df.mean(axis=1),
}
combined = pd.DataFrame(data)
combined.index = combined.index + t_intervention
combined = pd.merge(det_result, combined, left_index=True, right_index=True, how='outer')

for column in combined.columns:
    combined[column].dropna().plot(ax=ax[0])

ax[0].legend(combined.columns)

ax[1].set_xlabel('time (infectious periods)')
ax[0].set_ylabel('# Y')
ax[0].axvline(t_intervention, c='r', alpha=0.2)
ax[1].axvline(t_intervention, c='r', alpha=0.2)
ax[0].axvline(t_intervention+t_to_min, c='r', alpha=0.2)
ax[1].axvline(t_intervention+t_to_min, c='r', alpha=0.2)
ax[2].axvline(t_intervention, c='r', alpha=0.2)
ax[2].axvline(t_intervention+t_to_min, c='r', alpha=0.2)


R0s = [lambda_t_early_start(t)/mu for t in t_eval_early_start]

ax[1].plot(t_eval_early_start, R0s)
ax[1].set_ylabel('R0')
ax[1].axhline(1.0, c='r', linestyle='--', alpha=0.2)

ax[0].set_xlim(0, 15.0)

plt.savefig('./inhomogeneous.png', dpi=300)

ax_twin = ax[2]

cmap = {4:'seagreen', 2:'seagreen', 0: 'r', -2:'b', 1:'r'}
step_df_ib.plot.scatter(ax=ax_twin, x='t', y='step size', c=step_df_ib.status.map(cmap),alpha=0.5, ylim=[0., 0.2])
cmap = {4:'orange', 2:'orange', 0: 'r', -2:'b', 1:'r'}
step_df_homo.plot.scatter(ax=ax_twin, x='t', y='step size', c=step_df_homo.status.map(cmap),alpha=0.4, ylim=[0., 0.2])

plt.savefig('./inhomogeneous_with_step_size.png', dpi=300)

In [29]:
from scipy.integrate import simpson

In [ ]:
combined.columns

In [31]:
deterministic = combined['deterministic'].dropna()
#deterministic = deterministic[deterministic.index >= 1.0]
det_AUC = simpson(y=deterministic, x=deterministic.index)

In [ ]:
det_AUC

In [32]:
det_nub = combined['deterministic'].dropna()
det_nub = det_nub[det_nub.index <= 1.0]
nub_AUC = simpson(y=det_nub, x=det_nub.index)

In [33]:
conventional = combined['conventional $\\tau$'].dropna()
conventional_AUC = simpson(y=conventional, x=conventional.index) + nub_AUC

In [34]:
semi = combined['semi-inhomogeneous $\\tau$'].dropna()
semi_AUC = simpson(y=semi, x=semi.index) + nub_AUC

In [ ]:
semi_AUC

In [35]:
implicit = combined['implicit $\\tau$'].dropna()
implicit_AUC = simpson(y=implicit, x=implicit.index) + nub_AUC

In [ ]:
implicit_AUC

In [ ]:
(det_AUC - conventional_AUC)/det_AUC

In [ ]:
(det_AUC - semi_AUC)/det_AUC

In [ ]:
(det_AUC - implicit_AUC)/det_AUC